## Preliminary Check for 'review.json'

In [2]:
import json

review = "data/review.json"

with open(review, 'r') as f:
    for line_number, line in enumerate(f, start=1):
        try:
            obj = json.loads(line)
            print(obj.keys())
            break
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON on line {line_number}: {e}")


dict_keys(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date'])


## Check 'business.json'

In [5]:
import pandas as pd

business = pd.read_json("data/business.json", lines=True)
print(business.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150346 non-null  object 
 1   name          150346 non-null  object 
 2   address       150346 non-null  object 
 3   city          150346 non-null  object 
 4   state         150346 non-null  object 
 5   postal_code   150346 non-null  object 
 6   latitude      150346 non-null  float64
 7   longitude     150346 non-null  float64
 8   stars         150346 non-null  float64
 9   review_count  150346 non-null  int64  
 10  is_open       150346 non-null  int64  
 11  attributes    136602 non-null  object 
 12  categories    150243 non-null  object 
 13  hours         127123 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 16.1+ MB
None


## Filter the business data of FL & save to csv file

In [4]:
import csv

input_filename = 'data/business.json'
output_filename = 'data/florida_businesses.csv'

with open(input_filename, 'r') as infile, open(output_filename, 'w', newline='', encoding='utf-8') as outfile:
    csv_writer = None
    for line in infile:
        try:
            business = json.loads(line)
            if business['state'] == 'FL':
                if csv_writer is None:
                    csv_writer = csv.DictWriter(outfile, fieldnames=business.keys())
                    csv_writer.writeheader()
                csv_writer.writerow(business)
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {e}")
        except KeyError as e:
            print(f"Missing expected key: {e}")

print(output_filename, "saved")

data/florida_businesses.csv saved


## Filter the review data of FL & save

In [6]:
import pandas as pd
import itertools

florida_businesses_filename = 'data/florida_businesses.csv'
reviews_file = 'data/review.json'
output_file = 'data/florida_reviews.csv'

florida_businesses = pd.read_csv(florida_businesses_filename)
florida_business_ids = set(florida_businesses['business_id'])

chunk_size = 10000  
with open(reviews_file, 'r') as infile, open(output_file, 'w', newline='', encoding='utf-8') as outfile:
    header_written = False
    
    while True:
        lines = list(itertools.islice(infile, chunk_size))
        if not lines:
            break
        reviews = [json.loads(line) for line in lines]
        
        florida_reviews = [review for review in reviews if review['business_id'] in florida_business_ids]
        
        if florida_reviews:
            df = pd.DataFrame(florida_reviews)
            if not header_written:
                df.to_csv(outfile, index=False, mode='w')
                header_written = True
            else:
                df.to_csv(outfile, index=False, mode='a', header=False)
print(output_filename, "saved")

data/florida_businesses.csv saved


## Merge the business data & review data

In [7]:
florida_reviews_filename = 'data/florida_reviews.csv'
florida_businesses_filename = 'data/florida_businesses.csv'
output_filename = 'data/merged_florida_data.csv'

florida_reviews = pd.read_csv(florida_reviews_filename)
print("Florida reviews. Number of rows:", len(florida_reviews))

florida_businesses = pd.read_csv(florida_businesses_filename)
print("Florida businesses. Number of rows:", len(florida_businesses))

merged_data = pd.merge(florida_reviews, florida_businesses, on='business_id', how='left')
print("Datasets merged. Number of rows in merged dataset:", len(merged_data))

merged_data.to_csv(output_filename, index=False)
print("Merged data saved to", output_filename)

Florida reviews. Number of rows: 1161545
Florida businesses. Number of rows: 26330
Datasets merged. Number of rows in merged dataset: 1161545
Merged data saved to data/merged_florida_data.csv


## Filter the hills 

In [8]:
hill_businesses = pd.read_csv('data/hill_business.csv')
florida_business_ids = set(hill_businesses['business_id'])
reviews_iter = pd.read_csv('data/merged_florida_data.csv', chunksize=10000)

first_chunk = True
for chunk in reviews_iter:
    florida_reviews_chunk = chunk[chunk['business_id'].isin(florida_business_ids)]
    
    if not florida_reviews_chunk.empty:
        if first_chunk:
            florida_reviews_chunk.to_csv('data/merged_Tampa.csv', mode='w', index=False)
            first_chunk = False
        else:
            florida_reviews_chunk.to_csv('data/merged_Tampa.csv', mode='a', index=False, header=False)

In [10]:
file_path = 'data/merged_Tampa.csv'


df = pd.read_csv(file_path, nrows=0)  


print(df.columns.tolist())

['review_id', 'user_id', 'business_id', 'stars_x', 'useful', 'funny', 'cool', 'text', 'date', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars_y', 'review_count', 'is_open', 'attributes', 'categories', 'hours']


## Get the restaurant 

In [11]:
file_path = 'data/merged_Tampa.csv'
new_file_path = 'data/Tampa_restaurants.csv'

other_columns = [
    'review_id', 'user_id', 'business_id', 'stars_x', 'useful', 'funny', 'cool',
    'text', 'date', 'name', 'address', 'city', 'state', 'postal_code',
    'latitude', 'longitude', 'stars_y', 'review_count', 'is_open', 'categories', 'hours'
]


chunksize = 10 ** 5  
filtered_chunks = [] 

for chunk in pd.read_csv(file_path, chunksize=chunksize, usecols=['categories'] + other_columns):

    filtered_chunk = chunk[chunk['categories'].str.contains('restaurant', case=False, na=False)]
    filtered_chunks.append(filtered_chunk)


filtered_data = pd.concat(filtered_chunks)

filtered_data.to_csv(new_file_path, index=False)

print(new_file_path,'saved')

data/Tampa_restaurants.csv saved


## Get the Mexican restaurant data

In [13]:
data = pd.read_csv("data/Tampa_restaurants.csv")
mexican_count = data['categories'].str.contains('Mexican', case=False, na=False).sum()
print(f'Number of rows with Mexican in categories: {mexican_count}')
mexican_restaurants = data[data['categories'].str.contains('Mexican', case=False, na=False)]
mexican_restaurants.to_csv('data/Tampa_mexican_restaurants.csv', index=False)
tmr = pd.read_csv("data/Tampa_mexican_restaurants.csv")
print(tmr.info())

Number of rows with Mexican in categories: 34608
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34608 entries, 0 to 34607
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   review_id     34608 non-null  object 
 1   user_id       34608 non-null  object 
 2   business_id   34608 non-null  object 
 3   stars_x       34608 non-null  float64
 4   useful        34608 non-null  int64  
 5   funny         34608 non-null  int64  
 6   cool          34608 non-null  int64  
 7   text          34608 non-null  object 
 8   date          34608 non-null  object 
 9   name          34608 non-null  object 
 10  address       34572 non-null  object 
 11  city          34608 non-null  object 
 12  state         34608 non-null  object 
 13  postal_code   34608 non-null  float64
 14  latitude      34608 non-null  float64
 15  longitude     34608 non-null  float64
 16  stars_y       34608 non-null  float64
 17  review_count  34608 

## Filter the non-English 'text'

In [15]:
from langdetect import detect, LangDetectException

file_path = 'data/Tampa_mexican_restaurants.csv'

non_english_count = 0

def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return 'unknown'


chunksize = 10 ** 5  

for chunk in pd.read_csv(file_path, chunksize=chunksize, usecols=['text']):
    chunk_languages = chunk['text'].apply(detect_language)
    non_english_count += (chunk_languages != 'en').sum()

print(f'non-English text entries: {non_english_count}')

non-English text entries: 101


In [16]:
file_path = 'data/Tampa_mexican_restaurants.csv'
cleaned_file_path = 'data/cleaned_data.csv'

chunksize = 10 ** 5  
cleaned_chunks = []  

for chunk in pd.read_csv(file_path, chunksize=chunksize):
    def is_english(text):
        try:
            return detect(text) == 'en'
        except LangDetectException:
            return False

    chunk['is_english'] = chunk['text'].apply(is_english)
    cleaned_chunk = chunk[chunk['is_english']]
    cleaned_chunks.append(cleaned_chunk.drop(columns='is_english')) 

cleaned_data = pd.concat(cleaned_chunks)

cleaned_data.to_csv(cleaned_file_path, index=False)

print('cleaned_data saved')

cleaned_data saved


In [18]:
df = pd.read_csv("data/cleaned_data.csv")
df.head(5)

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,...,city,state,postal_code,latitude,longitude,stars_y,review_count,is_open,categories,hours
0,cCs7yPSyk8NdA-Ufoz_7hw,FodM8aoGMQO2zsQCQxBTYQ,v5ktgWMAARaczTMh2rAJKg,2.0,0,0,0,The service here has gone down. We used to go ...,2017-12-27 02:57:57,San Jose Mexican Restaurant,...,Lutz,FL,33558.0,28.189335,-82.541048,3.0,155,1,"Restaurants, Mexican, Salad","{'Monday': '11:0-21:30', 'Tuesday': '11:0-21:3..."
1,3lwAyxwk0zxKI8JkINtXXQ,BlJg3Lj8wRBLs71LmAsjEQ,3yI-7C8JzbG-ZaSqAGHMAQ,4.0,1,0,0,What a hidden gem of a restaurant. While ever...,2012-07-16 19:54:00,TacoSon,...,Tampa,FL,33617.0,28.017024,-82.393013,4.5,453,1,"Restaurants, Mexican","{'Monday': '7:0-21:0', 'Tuesday': '7:0-21:0', ..."
2,Uz94ISY4REBG0Yeuoszcgg,DrD4gyOQ3iOXWZUej153Ng,FSTmzo0beEVMRMSArcp4fg,2.0,0,0,0,"First visit tonight. Nice, clean place. Reason...",2015-11-19 00:30:41,Lime Fresh Mexican Grill,...,Tampa,FL,33607.0,27.959014,-82.524408,3.0,106,0,"Mexican, Restaurants","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
3,Naht5FrBGZDFGjwCQ-DyMw,QKoP7XabOXXWIkTwvYyCiA,xGUAa3xa8KsmbolC3XboQg,5.0,0,0,0,Xtreme Tacos has become one of my favorite pla...,2017-10-02 16:29:35,Xtreme Tacos,...,Tampa,FL,33604.0,28.025477,-82.457399,4.5,193,0,"Mexican, Tacos, Food, Restaurants, Food Trucks...","{'Monday': '0:0-0:0', 'Tuesday': '16:0-23:0', ..."
4,rsmwGEzqhbz49kUP62oT-w,xHISiVzIR4Bb1YP_SqvRkg,KP5OncF2jhT7_J1phHPPww,3.0,3,0,1,Visited last night with a group of six. Ambian...,2015-04-15 12:46:42,Besito Mexican Restaurant,...,Tampa,FL,33609.0,27.946250,-82.526568,4.0,1092,1,"Mexican, Restaurants, Salad, Food, Bars, Night...","{'Monday': '11:30-22:0', 'Tuesday': '11:30-22:..."


In [19]:
print(df.columns.tolist())
print(len(df))

['review_id', 'user_id', 'business_id', 'stars_x', 'useful', 'funny', 'cool', 'text', 'date', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars_y', 'review_count', 'is_open', 'categories', 'hours']
34512


## Drop the 'business star' column

In [20]:
df = pd.read_csv('data/cleaned_data.csv') 

df.drop('stars_x', axis=1, inplace=True)

df.rename(columns={'stars_y': 'stars'}, inplace=True)

df.to_csv('data/Tampa_MR.csv', index=False) 

## Data final review

In [22]:
ds = pd.read_csv('data/Tampa_MR.csv')
ds.head(5)
print(ds.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34512 entries, 0 to 34511
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   review_id     34512 non-null  object 
 1   user_id       34512 non-null  object 
 2   business_id   34512 non-null  object 
 3   useful        34512 non-null  int64  
 4   funny         34512 non-null  int64  
 5   cool          34512 non-null  int64  
 6   text          34512 non-null  object 
 7   date          34512 non-null  object 
 8   name          34512 non-null  object 
 9   address       34476 non-null  object 
 10  city          34512 non-null  object 
 11  state         34512 non-null  object 
 12  postal_code   34512 non-null  float64
 13  latitude      34512 non-null  float64
 14  longitude     34512 non-null  float64
 15  stars         34512 non-null  float64
 16  review_count  34512 non-null  int64  
 17  is_open       34512 non-null  int64  
 18  categories    34512 non-nu

In [1]:
import pandas as pd
df = pd.read_csv("data/hill_business.csv")
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12035 entries, 0 to 12034
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    12035 non-null  int64  
 1   business_id   12035 non-null  object 
 2   name          12035 non-null  object 
 3   address       11616 non-null  object 
 4   city          12035 non-null  object 
 5   state         12035 non-null  object 
 6   postal_code   12032 non-null  float64
 7   latitude      12035 non-null  float64
 8   longitude     12035 non-null  float64
 9   stars         12035 non-null  float64
 10  review_count  12035 non-null  int64  
 11  is_open       12035 non-null  int64  
 12  categories    12026 non-null  object 
 13  closed        2799 non-null   object 
dtypes: float64(4), int64(3), object(7)
memory usage: 1.3+ MB
None


In [4]:
import pandas as pd

df = pd.read_csv("data/Tampa_mexican_restaurants.csv")

df.columns.tolist()

['review_id',
 'user_id',
 'business_id',
 'stars_x',
 'useful',
 'funny',
 'cool',
 'text',
 'date',
 'name',
 'address',
 'city',
 'state',
 'postal_code',
 'latitude',
 'longitude',
 'stars_y',
 'review_count',
 'is_open',
 'categories',
 'hours']